In [1]:
import os
import sys

os.environ['JAVA_HOME'] = r"C:\Program Files\Java\jdk1.8.0_144"
os.environ['HADOOP_HOME'] = r"C:\Users\Asus\OneDrive\Desktop\pyspark\hadoop"

sys.path.append(os.environ['HADOOP_HOME'] + "\\bin")
os.environ['PATH'] += ";" + os.environ['JAVA_HOME'] + "\\bin"
os.environ['PATH'] += ";" + os.environ['HADOOP_HOME'] + "\\bin"

import findspark
findspark.init()

from pyspark.sql import SparkSession

In [2]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("PySpark-Jupyter-Demo") \
    .master("local[*]") \
    .getOrCreate()

In [3]:
# Test the setup
data = [("Kia", "Niro", 2025), ("Toyota", "Rav4", 2019), ("BYD", "Atto3", 2024)]
df = spark.createDataFrame(data, ["Car", "Model", "Year"])
df.show()

+------+-----+----+
|   Car|Model|Year|
+------+-----+----+
|   Kia| Niro|2025|
|Toyota| Rav4|2019|
|   BYD|Atto3|2024|
+------+-----+----+



In [4]:
df_lines = spark.read.format("com.databricks.spark.csv").options(header='true')\
 .option("delimiter", ",").option("quote", '"')\
 .option("escape", '"').option("multiLine", True)\
 .load("/Users/Asus/OneDrive/Desktop/pyspark/simpsons/simpsons_script_lines.csv")

In [5]:
df_lines.createOrReplaceTempView("lines_table")

In [6]:
## print lines file schema
df_lines.printSchema()

root
 |-- id: string (nullable = true)
 |-- episode_id: string (nullable = true)
 |-- number: string (nullable = true)
 |-- raw_text: string (nullable = true)
 |-- timestamp_in_ms: string (nullable = true)
 |-- speaking_line: string (nullable = true)
 |-- character_id: string (nullable = true)
 |-- location_id: string (nullable = true)
 |-- raw_character_text: string (nullable = true)
 |-- raw_location_text: string (nullable = true)
 |-- spoken_words: string (nullable = true)
 |-- normalized_text: string (nullable = true)
 |-- word_count: string (nullable = true)



In [7]:
## print all fields for first 5 rows
spark.sql("SELECT * from lines_table").show(5)

+----+----------+------+--------------------+---------------+-------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|  id|episode_id|number|            raw_text|timestamp_in_ms|speaking_line|character_id|location_id|  raw_character_text|   raw_location_text|        spoken_words|     normalized_text|word_count|
+----+----------+------+--------------------+---------------+-------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|9549|        32|   209|Miss Hoover: No, ...|         848000|         true|         464|          3|         Miss Hoover|Springfield Eleme...|No, actually, it ...|no actually it wa...|        31|
|9550|        32|   210|Lisa Simpson: (NE...|         856000|         true|           9|          3|        Lisa Simpson|Springfield Eleme...|Where's Mr. Bergs...| wheres mr bergstrom|         3|
|9551|        32|   

In [8]:
## print the shortest 10 script lines:
spark.sql("SELECT int(word_count) As wc, raw_text FROM lines_table WHERE TRY_CAST(word_count AS INT) > 0 ORDER BY wc").show(10,False)

+---+---------------------------------+
|wc |raw_text                         |
+---+---------------------------------+
|1  |Homer Simpson: Oh.               |
|1  |Homer Simpson: And?              |
|1  |Bart Simpson: Lewis?             |
|1  |Homer Simpson: (SHOCKED) Me?     |
|1  |Wendell Borton: Yayyyyyyyyyyyyyy!|
|1  |Lisa Simpson: Baboon!            |
|1  |Lisa Simpson: Yeah.              |
|1  |Lisa Simpson: No!                |
|1  |Homer Simpson: Oh.               |
|1  |Homer Simpson: Nuts.             |
+---+---------------------------------+
only showing top 10 rows



In [9]:
## count the scenes that took more than 10 minutes
spark.sql("SELECT count(*) FROM lines_table WHERE TRY_CAST(timestamp_in_ms AS INT) > (10 * 60* 1000)").show()

+--------+
|count(1)|
+--------+
|   86678|
+--------+



In [10]:
## print the first 50 locations that has the word "Springfield" in them , ignoring letters case.
df_locations = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("simpsons/simpsons_locations.csv")
df_locations.createOrReplaceTempView("locations_table")

spark.sql("SELECT name FROM locations_table WHERE lower(name) LIKE '%springfield%' LIMIT 50").show(50, False)

+-----------------------------------------------+
|name                                           |
+-----------------------------------------------+
|Springfield Elementary School                  |
|Springfield Mall                               |
|Springfield Nuclear Power Plant                |
|Springfield Downs Dog Track                    |
|SPRINGFIELD DOWNS                              |
|SPRINGFIELD DOWN                               |
|SPRINGFIELD DOWNS PARKING LOT                  |
|Springfield Elementary School Playground       |
|Springfield Town Hall                          |
|Springfield Elementary School Hallway          |
|Springfield Elementary School Cafeteria        |
|Springfield Retirement Castle                  |
|Grampa's Room at Springfield Retirement Castle |
|Springfield Library                            |
|Springfield Elementary School Yard             |
|First Church of Springfield                    |
|Springfield Elementary School Library          |


In [11]:
## print 20 quotes that are located in any place that has Jerusalem in its name.
## Note that Jerusalem may appear in any case and in any part of the location name.
## Use JOIN for this query on another table
spark.sql("SELECT l.spoken_words as Quote, loc.name as Location FROM lines_table l JOIN locations_table loc ON l.location_id = loc.id WHERE lower(loc.name) LIKE '%jerusalem%' LIMIT 20").show(20, False)


+---------------------------------------------------------------------------------------------------------------------------------------------+----------------------+
|Quote                                                                                                                                        |Location              |
+---------------------------------------------------------------------------------------------------------------------------------------------+----------------------+
|NULL                                                                                                                                         |JERUSALEM ROYAL PALACE|
|I'm bored. Send in my jester.                                                                                                                |JERUSALEM ROYAL PALACE|
|Hey, hey, King David! Ha, ha, how ya doin'? Now I'm not saying Jezebel's easy, but before she moved to Sodom, it was known for its pottery.  |JERUSALEM ROYAL PALACE

In [12]:
## print first 20 most used locations with the count of lines spoken in them.
## use GROUP BY for that
spark.sql("SELECT loc.name, COUNT(*) as line_count FROM lines_table l JOIN locations_table loc ON l.location_id = loc.id GROUP BY loc.name ORDER BY line_count DESC LIMIT 20").show(20, False)

+-------------------------------+----------+
|name                           |line_count|
+-------------------------------+----------+
|Simpson Home                   |35059     |
|Springfield Elementary School  |7092      |
|Moe's Tavern                   |4628      |
|Springfield Nuclear Power Plant|3594      |
|Kwik-E-Mart                    |1476      |
|First Church of Springfield    |1416      |
|Simpson Living Room            |1378      |
|Springfield Street             |1334      |
|Springfield                    |1314      |
|Simpson Car                    |1239      |
|Flanders Home                  |1166      |
|Street                         |1124      |
|Springfield Town Hall          |1103      |
|Springfield Retirement Castle  |1049      |
|Burns Manor                    |998       |
|Springfield Mall               |833       |
|Simpson Kitchen                |816       |
|Courtroom                      |813       |
|Bart's Treehouse               |767       |
|Bart's Be

In [13]:
## find the seasons in which the average imdb rating was the highest.
## Print the seasons number, the number of episodes in each one and the average rating
## in a descending order from highest average rating to lowest.

df_episodes = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("simpsons/simpsons_episodes.csv")
df_episodes.createOrReplaceTempView("episodes_table")

spark.sql("SELECT season, COUNT(*) as episode_count, ROUND(AVG(imdb_rating), 2) as average_rating FROM episodes_table GROUP BY season ORDER BY average_rating DESC").show()

+------+-------------+--------------+
|season|episode_count|average_rating|
+------+-------------+--------------+
|     5|           22|          8.34|
|     7|           25|          8.32|
|     6|           25|          8.31|
|     4|           22|          8.27|
|     8|           25|          8.22|
|     3|           24|          8.15|
|     2|           22|          8.04|
|     9|           25|          7.84|
|     1|           13|          7.81|
|    10|           23|          7.57|
|    12|           21|          7.36|
|    11|           22|          7.29|
|    13|           22|          7.14|
|    14|           22|          7.08|
|    15|           22|          7.05|
|    16|           21|          7.04|
|    18|           22|           7.0|
|    19|           20|          6.94|
|    20|           21|           6.9|
|    17|           22|          6.86|
+------+-------------+--------------+
only showing top 20 rows



In [14]:
## Find the 10 characters with the highest number of spoken lines throughout the series.
## Print the character name and the count of lines in descending order.
df_characters = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("simpsons/simpsons_characters.csv")
df_characters.createOrReplaceTempView("characters_table")

spark.sql("SELECT c.name, COUNT(*) as lines_count FROM lines_table l JOIN characters_table c ON l.character_id = c.id GROUP BY c.name ORDER BY lines_count DESC LIMIT 10").show(10, False)

+-------------------+-----------+
|name               |lines_count|
+-------------------+-----------+
|Homer Simpson      |30108      |
|Marge Simpson      |14265      |
|Bart Simpson       |13969      |
|Lisa Simpson       |11641      |
|C. Montgomery Burns|3207       |
|Moe Szyslak        |2864       |
|Seymour Skinner    |2443       |
|Ned Flanders       |2145       |
|Grampa Simpson     |1957       |
|Milhouse Van Houten|1911       |
+-------------------+-----------+

